In [1]:
# Implementation of the Bottom Up Network with two Subversions: 
# BK with different Kernelsize and BF with different feature size
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehot, onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON
using LinearAlgebra

using MAT # needs installation of Pkg.add("MAT")
using PyPlot # pip install Matplotlib Pkg.add("PyPlot")

In [4]:
# PARAMETERS
# ----------
batch_size = 100
n = 5
c = 1
alpha = 10^-4
beta = 0.5
lambda = 0.0005

kernelsize = (3, 3)
featuresize = 32

momentum = 0.9
# TODO drop the learning rate according to the paper
learningRate = 0.01

0.01

In [23]:
# load and process training data into batches
# Data order in the .mat file 
# images: N_samples x 32 x 32 x 1
# targets N_samples x 1
# bin_targets: N_samples x 10

file = matopen("../digitclutter/src/light_debris/light_debris_with_debris.mat")
images = read(file, "images")
targets = read(file, "targets")
bin_targets = read(file, "binary_targets")
close(file) 
# rearrange the images array so it matches the convention of Flux width x height x channels x batchsize(Setsize)
images = permutedims(images, (2, 3, 4, 1))
# rearrange binary targets: targetarray(10) x batchsize(Setsize)
bin_targets = permutedims(bin_targets, (2, 1))

# Display one sample of the images
# matshow(dropdims(images[:,:,:,10], dims=3), cmap=PyPlot.cm.gray, vmin=0, vmax=255)#, cmap = gray, vmin=0, vmax=255)

# SOURCE: https://github.com/FluxML/model-zoo/blob/master/vision/mnist/conv.jl
# Bundle images together with labels and group into minibatches
# X should be of size Width x Height x 1 x Samples
# Y should be of size Samples x 1
# X_batch is 32 x 32 x 1 x batchsize
# Y_batch is 10 x batchsize
function make_minibatch(X, Y, idxset)
    # ... expands the inputarguments of the array construction to all sizes of X: size(X[1]), size(X[2]), size(X[3])
    X_batch = Array{Float32}(undef, size(X, 1), size(X, 2), 1, length(idxset))
    Y_batch = Array{Float32}(undef, 10, length(idxset))
    for i in 1:length(idxset)
        
        X_batch[:, :, :, i] = Float32.(X[:, :, :, idxset[i]])
        Y_batch[:, i] = Y[:, idxset[i]]
    end
    #Y_batch = onehotbatch(Y[idxset], 0:9) 
    return (X_batch, Y_batch)
end

# create the actual batches
idxsets = partition(1:size(images, 4), batch_size)
train_set = [make_minibatch(images, bin_targets, i) for i in idxsets];

# train_set is a set of tuples (x_batch, y_batch) - first bracket -> which tupel, second bracket -Y what of the tupel
train_set[1][1]

32×32×1×50 Array{Float32,4}:
[:, :, 1, 1] =
 119.0  119.0  119.0  119.0  119.0  …  119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0  119.0     119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0  119.0     119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0  119.0     119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0  119.0     119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0  119.0  …  119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0  119.0     119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0  119.0     119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0  119.0     119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0    0.0     119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0   25.0  …  119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0  110.0     119.0  119.0  119.0  119.0  119.0
 119.0  119.0  119.0  119.0  122.0     119.0  119.0  119.0  119.0  119.0
   ⋮   

In [16]:
# Input is either 32x32x32xN or 16x16x32xN
function localResponseNorm(x)
    w = zeros(size(x))
    for k = 1:size(x,3)
        # calculate the boundaries of the sum
        # ATTENTION: this will sum n+1 Elements
        lower = Int32(max(1, round(k-n/2)))
        upper = Int32(min(size(x,3), round(k+n/2)))
        
        # square and sum 
        # w = map((x) -> x^2, x[:, :, lower:upper, :])
        norm = Tracker.data(x[:, :, lower:upper, :])
        norm = norm .^ 2
        # reduce to one 32x32x1xN or 16x16x1xN matrix
        norm = sum(norm, dims=3)
        norm = norm .* alpha
        norm = norm .+ c
        norm = norm .^ (-beta)
        w[:, :, k, :] = norm
    end   
    # a tracked array multiplied with a non tracked array returns a tracked array  
    x = w .* x
    return x
end

localResponseNorm (generic function with 1 method)

In [17]:
function loss(x, y)
    y_hat = model(x)
    return crossentropy(y_hat, y) + lambda * sum(norm, params(model))
end

loss (generic function with 1 method)

In [18]:
@info("Constructing model...")
model = Chain(
    # HIDDEN LAYER 1
    # Input Image 32x32x1xN
    Conv(kernelsize, 1=>featuresize, pad=(1,1), relu),
    # local response normalization
    x -> localResponseNorm(x),
    MaxPool((2, 2), stride=(2, 2)),
    
    # HIDDEN LAYER 2
    # Input Image 16x16x32xN
    Conv(kernelsize, featuresize=>featuresize, pad=(1,1), relu),
    # local response normalization
    x -> localResponseNorm(x),
    MaxPool((16, 16), stride=(1, 1)),
    # reshape to 32xN
    x -> reshape(x, :, size(x, 4)),
    Dense(32, 10, σ),
)

# test the model (precomile it??)
model(rand(32, 32, 1, 1))

┌ Info: Constructing model...
└ @ Main In[18]:1


Tracked 10×1 Array{Float32,2}:
 0.30785903f0
 0.5292328f0 
 0.63956386f0
 0.59460783f0
 0.07974327f0
 0.52473676f0
 0.80157465f0
 0.10636985f0
 0.51938564f0
 0.38682187f0

In [24]:
@info("Training...")
optimizer = Momentum(learningRate, momentum)
# using the momentum update rule

label = zeros(10, 2)
label[3, 1] = 1
label[4, 2] = 1

data = [(rand(32,32,1,2), label), (rand(32,32,1,2), label)]
# training probably works but pixelwise normalization is missing 
# so loss gets NaN
Flux.train!(loss, params(model), data, optimizer)

┌ Info: Training...
└ @ Main In[24]:1
